In [36]:
import sys
import os
import torch
sys.path.append("C:/Users/yuuta/Documents/fashion")
from utils.util import open_json
from utils.util import is_target_category
import glob
import random
from PIL import Image
from utils.util import get_category


In [71]:
import importlib
import utils
importlib.reload(utils.util)
importlib.reload(utils.versatility)

<module 'utils.versatility' from 'C:\\Users/yuuta/Documents/fashion\\utils\\versatility.py'>

In [4]:
if torch.cuda.is_available():
    device = torch.device("cuda")  # GPUデバイスを取得
else:
    device = torch.device("cpu")  # CPUデバイスを取得

In [5]:
filepaths = glob.glob(
    f"C:/Users/yuuta/Documents/fashion/data/test/**/*.json"
)

In [60]:
id_to_vector_versatility = open_json('C:/Users/yuuta/Documents/fashion/experiments/data/id_to_vector_versatility.json')

In [61]:
items = {"tops": [], "bottoms": [], "shoes": []}
for fp in filepaths:
    json_dict = open_json(fp)
    if json_dict == None:
        continue
    for item in list(filter(is_target_category, json_dict["items"])):
        category = get_category(item)
        if category == None:
            continue
        items[category].append(str(item['itemId']))

In [62]:
categories = ['tops', 'bottoms', 'shoes']
# 枚数
T = 3

In [78]:
from utils.save_image import save_combined_image
from utils.versatility import calc_increase_v
dataset = {
    'tops':items["tops"],
    'bottoms':items["bottoms"],
    'shoes':items["shoes"],
}
N = 3

TEST_KIND = 'ver'
for cat in categories:
    dir_path = f'C:/Users/yuuta/Documents/fashion/experiments/questionnaire/images/versatility/{TEST_KIND}/{cat}/'
    if not os.path.exists(dir_path):
        os.makedirs(dir_path)
    for i in range(N):
        ver_score = []
        dir_path = f'C:/Users/yuuta/Documents/fashion/experiments/questionnaire/images/versatility/{TEST_KIND}/{cat}/test{i+1}'
        if not os.path.exists(dir_path):
            os.makedirs(dir_path)
        anchor = random.choice(dataset[cat])
        anchor_v = id_to_vector_versatility[anchor]
        anchor_cover_clusters = calc_increase_v(torch.tensor(anchor_v))
        anchor_img = Image.open(f"C:/Users/yuuta/Documents/fashion/data/images/{anchor}.jpg")
        anchor_img.save(f'C:/Users/yuuta/Documents/fashion/experiments/questionnaire/images/versatility/{TEST_KIND}/{cat}/test{i+1}/anchor.png')
        for item in dataset[cat]:
            v = id_to_vector_versatility[item]
            cover_clusters = calc_increase_v(torch.tensor(v))
            # d = torch.norm(torch.tensor(v) - torch.tensor(anchor_v), p=2)
            d = len(anchor_cover_clusters & cover_clusters)
            score = d
            ver_score.append((item, score))
        ranking = sorted(ver_score, key=lambda x: x[1], reverse=True)
        divs = [0, len(dataset[cat]) // 4, len(dataset[cat]) // 2, (len(dataset[cat]) * 3) // 4]
        for div, name in zip(divs, ['0%', '25%', '50%', '75%']):
            for rank in range(div, div + 10, 1):
                item = ranking[rank][0]
                item_img = Image.open(f"C:/Users/yuuta/Documents/fashion/data/images/{item}.jpg")
                dir_path = f'C:/Users/yuuta/Documents/fashion/experiments/questionnaire/images/versatility/{TEST_KIND}/{cat}/test{i+1}/{name}/'
                if not os.path.exists(dir_path):
                    os.makedirs(dir_path)
                item_img.save(f'C:/Users/yuuta/Documents/fashion/experiments/questionnaire/images/versatility/{TEST_KIND}/{cat}/test{i+1}/{name}/{rank}.png')

In [ ]:
# ranking = sorted(ver_score, key=lambda x: x[1])
# item_ids = list(map(lambda x: x[0], ranking[:30]))
# item_ids[0] = anchor
# save_combined_image(item_ids, f'C:/Users/yuuta/Documents/fashion/experiments/questionnaire/images/versatility/sample_ver_{cat}.png', W=10)
        

In [73]:
ranking = sorted(ver_score, key=lambda x: x[1], reverse=True)

In [75]:
ranking[-20:]

[('14543191', 389),
 ('40117067', 388),
 ('32179679', 386),
 ('16180526', 385),
 ('31286538', 385),
 ('38402518', 384),
 ('33392977', 383),
 ('41863393', 382),
 ('36276221', 380),
 ('12757608', 379),
 ('38383315', 376),
 ('12874847', 375),
 ('41706382', 374),
 ('42518355', 372),
 ('31401974', 371),
 ('31401974', 371),
 ('37658874', 369),
 ('41790650', 368),
 ('41790650', 368),
 ('16908952', 361)]